In [2]:
%matplotlib inline
from matplotlib import pyplot as plt
import velocyto as vcy
import loompy
import anndata
import numpy as np
import scanpy.api as sc
import scipy.io
import pandas as pd

sc.settings.set_figure_params(dpi=100)

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Creating a loom object/file with loompy:
A loom object has three components
* Data matrix, either numpy ndarray or scipy sparse matrix
* Dictionary of row attributes
* Dictionary of column attributes

If the data matrix is a N row x M col matrix, the row attribute dict must have M elements and the column attribute dict must have N elements.

In [3]:
input_dir = "/Users/samuelmorabito/Documents/kessenbrock_rotation/RNA_velocity/ind_5_6_7_aggr/filtered_gene_bc_matrices_mex/GRCh38/"
counts_matrix = scipy.io.mmread(input_dir + '/matrix.mtx').T.tocsc()
barcodes = np.array([line[:-1] for line in open(input_dir + '/barcodes.tsv')])
genes = np.array([line.split()[1] for line in open(input_dir + "/genes.tsv")])

In [4]:
#isolate entries from individual 5:
ind5_barcodes = np.array([b for b in barcodes if b.endswith("-1")])
ind5_filter = np.ravel(np.array([True if b.endswith("-1") else False for b in barcodes]))
ind5_counts_matrix = counts_matrix[ind5_filter]

In [5]:
col_attributes = {"genes": genes}
row_attributes = {"barcodes": ind5_barcodes}
loompy.create(
    "/Users/samuelmorabito/Documents/kessenbrock_rotation/RNA_velocity/loom_files/ind5.loom",
    ind5_counts_matrix,
    row_attributes,
    col_attributes
)

Some notes about creating loom files:

* `loompy.create()` does not return anything. To work with the .loom file, you need to use `loompy.connect("file.loom")`. 
* The [loompy documentation](https://linnarssonlab.org/loompy/apiwalkthrough/index.html) recommends using something like `with loom.connect("file.loom") as ds:`. Alternatively, if you do something like `ds = loom.connect("file.loom")` you also need something like `ds.close()` to close the connection to the loom file.
* Instead of `loompy.create()` you can use `loompy.new()` to create and connect simultaneously.

In [6]:
ind5_loom = loompy.connect("/Users/samuelmorabito/Documents/kessenbrock_rotation/RNA_velocity/loom_files/ind5.loom")

In [7]:
ind5_loom.close()

In [8]:
vlm = vcy.VelocytoLoom("../RNA_velocity/loom_files/ind5.loom")

AttributeError: '<class 'loompy.layer_manager.LayerManager'>' object has no attribute 'spliced'